In [20]:
import pandas as pd
import os
# 定义文件夹路径
folder_path = 'data'

def check_file_content_by_dir(file_path):
    for root,dirs,files in os.walk(file_path):
        for file in files:
            if  file.endswith(('.txt','.zip')):
                continue
            print(f"File: {os.path.join(root,file)}")
            try:
                df = pd.read_csv(os.path.join(root,file),encoding='gb18030',on_bad_lines='skip',header=None)
                
                
                # 打印第一条数据
                if not df.empty:
                    print("First row:", df.iloc[0].to_dict())
                else:
                    print("This file is empty.")
                
                print("-" * 40)
            except Exception as e:
                print(f"Could not read {file}: {e}")

check_file_content_by_dir('data')


File: data/.DS_Store
Could not read .DS_Store: 'gb18030' codec can't decode byte 0x80 in position 3131: illegal multibyte sequence
File: data/sougou_competition_data/user_tag_query.10W.TEST
Could not read user_tag_query.10W.TEST: 'gb18030' codec can't decode byte 0x80 in position 65767: illegal multibyte sequence
File: data/sougou_competition_data/user_tag_query.10W.TRAIN
Header: [0, 1, 2, 3]
First row: {0: '22DD920316420BE2DF8D6EE651BA174B\t1\t1\t4\t柔和双沟\t女生\t中财网首页 财经\thttp://pan.baidu.com/s/1plpjtn9\t周公解梦大全查询2345\t曹云金再讽郭德纲\t总裁大人行行好\t中财网第一财经传媒\t教师节全文\t男子砸毁15墓碑\t黄岩岛最新填海图\t引起的疲\t缘来未迟落跑甜心不好惹\t梁朝伟与替身同框\t笑傲江湖电视剧任贤齐\t小起名字女孩名字\t海运行李到堪培拉\t确定\t诱爱99天 司少的天价宝贝\t什么是遥控魔棒\t徽信表情动态搞笑图片\t教师节征文100字\t安微联通网上营业厅\t甜宠百分百:校草的萌萌未婚妻\t豪门重生之暖爱成婚\tnikehypershift和kd5哪个好看\t韭菜炒鸡蛋\t陈赫玩王者荣耀\t虎牙楚河\t三国演义小说txt下载\t威县欧派\t炒馍花怎么做好吃\t黄岩岛最新消息2016年\t中秋节诗句大全祝福\t教师节征文\t菜谱\t柔和双沟卖的怎么样\t七位数开奖结果\t以色列停车场坍塌\t天龙家庭农场\t7.22什么星座\t新旧约圣经和合本下载\t4π\twifi万能钥匙\t威灵仙图片\t临泉长官天龙家庭农场\t早安总统大人\t百合\t莲藕的做法\t花街\t无锡\t蚬壳胃散怎么吃\t触手忆寒\t中秋节的诗句\t孟州电信 电子发票\t鸡丝汤的做法

# 数据预处理

In [14]:
import csv

with open('data/sougou_competition_data/user_tag_query.10W.TRAIN', 'r', encoding='gb18030') as file:
    # 读取文件内容并按换行符分隔
    lines = file.read().splitlines()

raw_queries = []
for line in lines:
    query = line.split('\t')[4:]
    raw_queries.extend(query)

In [1]:
from tqdm import tqdm
# from autocorrect import Speller
from pycorrector import Corrector
corrector = Corrector()

correct_queries = []
count = 0
for query in tqdm(raw_queries):
    blob = corrector.correct(query)
    correct_queries.append(blob)
    if query != blob['target']:
        count += 1
        # print(f"Original: {query}")
        # print(f"Corrected: {blob}")
# print(count)




KeyboardInterrupt: 

In [8]:
    
import re
unqualified_pattern = re.compile(
    r'(http[s]?://\S+|www\.\S+)'          # 匹配 URL
    r'|(\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b)'  # 匹配邮箱
    r'|(^[0-9.]+$)'                        # 仅含数字或特殊字符
)
def is_valid_query(query):
    return not unqualified_pattern.search(query)

queries = list(filter(is_valid_query, raw_queries))


import jieba
import jieba.posseg as pseg

query_words = []
allowPOS = ['n']  # 允许的词性列表

from tqdm import tqdm

# 假设 allowPOS 和 query_words 已经定义
# allowPOS = set(...)  
# query_words = []

# 使用 tqdm 为查询列表添加进度条
for query in tqdm(queries, desc="Processing queries"):
    a_query_words = []
    # 使用带词性的分词
    words = pseg.cut(query)
    for word, flag in words:
        # 仅保留指定词性且长度大于等于 2 的词
        if (flag[0] in allowPOS) and len(word) >= 2:
            a_query_words.append(word)
    # 仅在分词后词数大于 1 时添加到 query_words
    if len(a_query_words) > 1:
        query_words.append(a_query_words)

    
# 保存数据到 CSV 文件
with open('processed_data.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(query_words)


Process SpawnPoolWorker-80482:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/lib/python3.12/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/opt/anaconda3/lib/python3.12/multiprocessing/queues.py", line 389, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'process_query' on <module '__main__' (<class '_frozen_importlib.BuiltinImporter'>)>
Process SpawnPoolWorker-80483:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs

KeyboardInterrupt: 

# 拆分出一个小数据集

In [1]:
import csv

# 原始文件路径
original_file_path = 'processed_data.csv'
# 新文件路径
new_file_path = 'small_processed_data.csv'

# 计算要抽取的行数
with open(original_file_path, 'r', encoding='utf-8') as file:
    total_lines = sum(1 for line in file)
    sample_size = max(1, total_lines // 200)  # 确保至少有一行

# 读取并写入前1/100的数据
with open(original_file_path, 'r', encoding='utf-8') as file, \
     open(new_file_path, 'w', newline='', encoding='utf-8') as new_file:
    reader = csv.reader(file)
    writer = csv.writer(new_file)
    
    for i, row in enumerate(reader):
        if i >= sample_size:
            break
        writer.writerow(row)

print(f"前 1/200 的数据已保存到 {new_file_path}")

前 1/200 的数据已保存到 small_processed_data.csv


# 竞争性关键词模型类

In [5]:
class CompKey:

    def __init__(self, name):
        # 初始化字典和变量
        self.name = name
        self.inters = {}         # 中介关键词字典
        self.comps = {}          # 竞争关键词字典
        self.inters_all = {}     # 每个中介关键词出现的总次数
        self.result = {}         # 存放最终竞争度排名结果
        self.keyword = ''        # 当前处理的种子关键词
        self.num_queries = 0     # 当前种子关键词的查询数量
        self.num_inters = 0      # 中介关键词数量
        self.threshold = 0       # 中介关键词过滤阈值
        self.comp_cache = {}     # 缓存关键词竞争度排名结果

    def initialize(self, keyword):
        """初始化关键词相关数据"""
        self.inters.clear()
        self.comps.clear()
        self.num_queries = 0
        self.num_inters = 0
        self.keyword = keyword

    def train(self, data, keywords: list, num_results: int = 20):
        """训练模型，对每个关键词进行竞争性分析"""
        for step, keyword in enumerate(keywords):
            print(f"Training keyword {step + 1}/{len(keywords)}: '{keyword}'")
            # 初始化
            self.initialize(keyword)
            # 筛选包含种子关键词的查询
            filtered_data = self.filter_query(data)
            # 分析中介关键词和竞争关键词
            self.analyze_inter(filtered_data)
            self.analyze_comp(data)
            # 计算竞争度并缓存结果
            self.comp_cache[keyword] = self.calculate_comp()[:num_results]
            print(f"Completed training for '{keyword}'")
    
    def save_to_disk(self, filename="comp_cache.json"):
        """将训练结果保存到磁盘上的文件中"""
        try:
            # 打开文件并以 JSON 格式写入 comp_cache
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(self.comp_cache, f, ensure_ascii=False, indent=4)
            print(f"Training results saved successfully to '{filename}'")
        except Exception as e:
            print(f"An error occurred while saving to disk: {e}")

    def predict(self, keyword):
        """预测指定关键词的竞争度排名"""
        return self.comp_cache.get(keyword, None)

    def filter_query(self, data):
        """筛选包含种子关键词的查询记录"""
        print("Filtering queries...")
        start_time = time.time()
        # 筛选出包含关键词的查询
        filtered_data = [query for query in data if self.keyword in query]
        self.num_queries = len(filtered_data)
        end_time = time.time()
        print(f"Got {self.num_queries} queries related to keyword '{self.keyword}'. Time taken: {end_time - start_time:.2f}s")
        return filtered_data

    def analyze_inter(self, data):
        """分析中介关键词"""
        print("Analyzing intermediate keywords (inters)...")
        start_time = time.time()
        for query in data:
            for word in query:
                if word != self.keyword:
                    self.dict_add(self.inters, word)
        inters_total = len(self.inters)
        # 过滤出现次数较少的中介关键词
        keys_to_delete = [key for key, count in self.inters.items() if count < self.threshold]  #TODO 这里的阈值需要调整
        for key in keys_to_delete:
            del self.inters[key]
        self.num_inters = len(self.inters)
        end_time = time.time()
        print(f"Got {self.num_inters}/{inters_total} inters after filtering. Time taken: {end_time - start_time:.2f}s")

    def analyze_comp(self, data):
        """分析竞争关键词"""
        print("Analyzing competing keywords (comps)...")
        start_time = time.time()
        with tqdm(total=self.num_inters, desc="Progress", unit="keyword") as pbar:
            for step, key in enumerate(self.inters):
                query_start_time = time.time()
                # 找到包含中介关键词的查询
                filtered_data = [query for query in data if key in query]
                self.inters_all[key] = len(filtered_data)
                for query in filtered_data:
                    for word in query:
                        if word != key and word != self.keyword:
                            self.dict_add(self.comps, (word, key))
                query_end_time = time.time()
                # 更新进度条
                pbar.set_postfix(time_per_keyword=f"{query_end_time - query_start_time:.2f}s")
                pbar.update(1)  # 每处理一个关键词，更新一次进度条
        end_time = time.time()
        print(f"Completed analyzing comps. Total comp-inter pairs: {len(self.comps)}. Time taken: {end_time - start_time:.2f}s")

    def calculate_comp(self):
        """计算竞争关键词的竞争度"""
        print("Calculating competition scores...")
        start_time = time.time()
        for comp, inter in self.comps:
            num = self.comps[(comp, inter)]
            value = self.result.get(comp, 0)
            # 跳过没有竞争关键词的中介关键词
            if self.inters_all[inter] == self.inters[inter]:
                continue  # 跳过该中介关键词
            
            # 计算竞争度分数
            self.result[comp] = value + (self.inters[inter] / self.num_queries) * (num / (self.inters_all[inter] - self.inters[inter]))
        
        end_time = time.time()
        print(f"Completed calculation of competition scores. Time taken: {end_time - start_time:.2f}s")
        return sorted(self.result.items(), key=lambda d: d[1], reverse=True)

    @staticmethod
    def dict_add(_dict, key):
        """增加字典中 key 的计数"""
        if key in _dict:
            _dict[key] += 1
        else:
            _dict[key] = 1

In [4]:
import json
from tqdm import tqdm
import time
from collections import defaultdict

class CompKeyLL:

    def __init__(self, name):
        # 初始化字典和变量
        self.name = name
        self.inters = {}         # 中介关键词字典
        self.comps = {}          # 竞争关键词字典
        self.inters_all = {}     # 每个中介关键词出现的总次数
        self.result = {}         # 存放最终竞争度排名结果
        self.keyword = ''        # 当前处理的种子关键词
        self.num_queries = 0     # 当前种子关键词的查询数量
        self.num_inters = 0      # 中介关键词数量
        self.threshold = 0       # 中介关键词过滤阈值
        self.comp_cache = {}     # 缓存关键词竞争度排名结果

    def initialize(self, keyword):
        """初始化关键词相关数据"""
        self.inters.clear()
        self.comps.clear()
        self.inters_all.clear()
        self.result.clear()
        self.num_queries = 0
        self.num_inters = 0
        self.keyword = keyword

    def train(self, data, keywords: list, num_results: int = 20):
        """训练模型，对每个关键词进行竞争性分析"""
        print("构建关键词到查询的映射（word_to_queries）...")
        word_to_queries = defaultdict(set)
        
        for i, query in enumerate(data):
            for word in set(query):  # 使用 set 来避免同一查询中重复的词
                word_to_queries[word].add(i)
        print("word_to_queries 构建完成。")
        num_indices = len(word_to_queries)
        print(f"Got {num_indices} unique words in the dataset.")
        print("前5个关键词及其查询索引:")
        for word, query_indices in list(word_to_queries.items())[:5]:
            print(f"关键词 '{word}' 出现在查询索引: {sorted(query_indices)}")

        for step, keyword in enumerate(keywords):
            print(f"Training keyword {step + 1}/{len(keywords)}: '{keyword}'")
            # 初始化
            self.initialize(keyword)
            # 筛选包含种子关键词的查询
            filtered_query_indices = word_to_queries.get(keyword, set())
            filtered_data = [data[i] for i in filtered_query_indices]
            self.num_queries = len(filtered_data)
            # 分析中介关键词和竞争关键词
            self.analyze_inter(filtered_data)
            self.analyze_comp(data, word_to_queries)
            # 计算竞争度并缓存结果
            self.comp_cache[keyword] = self.calculate_comp()[:num_results]
            print(f"Completed training for '{keyword}'")

    def save_to_disk(self, filename="comp_cache.json"):
        """将训练结果保存到磁盘上的文件中"""
        try:
            # 打开文件并以 JSON 格式写入 comp_cache
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(self.comp_cache, f, ensure_ascii=False, indent=4)
            print(f"Training results saved successfully to '{filename}'")
        except Exception as e:
            print(f"An error occurred while saving to disk: {e}")

    def predict(self, keyword):
        """预测指定关键词的竞争度排名"""
        return self.comp_cache.get(keyword, None)

    def analyze_inter(self, filtered_data):
        """分析中介关键词"""
        print("Analyzing intermediate keywords (inters)...")
        start_time = time.time()
        for query in filtered_data:
            for word in query:
                if word != self.keyword:
                    self.dict_add(self.inters, word)
        inters_total = len(self.inters)
        # 过滤出现次数较少的中介关键词
        keys_to_delete = [key for key, count in self.inters.items() if count < self.threshold]
        for key in keys_to_delete:
            del self.inters[key]
        self.num_inters = len(self.inters)
        end_time = time.time()
        print(f"Got {self.num_inters}/{inters_total} inters after filtering. Time taken: {end_time - start_time:.2f}s")

    def analyze_comp(self, data, word_to_queries):
        """分析竞争关键词"""
        print("Analyzing competing keywords (comps)...")
        start_time = time.time()
        with tqdm(total=self.num_inters, desc="Progress", unit="keyword") as pbar:
            for step, key in enumerate(self.inters):
                query_start_time = time.time()
                # 获取包含中介关键词的查询索引
                inter_query_indices = word_to_queries.get(key, set())
                self.inters_all[key] = len(inter_query_indices)
                for i in inter_query_indices:
                    query = data[i]
                    for word in query:
                        if word != key and word != self.keyword:
                            self.dict_add(self.comps, (word, key))
                query_end_time = time.time()
                # 更新进度条
                pbar.set_postfix(time_per_keyword=f"{query_end_time - query_start_time:.2f}s")
                pbar.update(1)  # 每处理一个关键词，更新一次进度条
        end_time = time.time()
        print(f"Completed analyzing comps. Total comp-inter pairs: {len(self.comps)}. Time taken: {end_time - start_time:.2f}s")

    def calculate_comp(self):
        """计算竞争关键词的竞争度"""
        print("Calculating competition scores...")
        start_time = time.time()
        for comp, inter in self.comps:
            num = self.comps[(comp, inter)]
            value = self.result.get(comp, 0)
            # 跳过没有竞争关键词的中介关键词
            if self.inters_all[inter] == self.inters[inter]:
                continue  # 跳过该中介关键词

            # 计算竞争度分数
            self.result[comp] = value + (self.inters[inter] / self.num_queries) * (num / (self.inters_all[inter] - self.inters[inter]))
        
        end_time = time.time()
        print(f"Completed calculation of competition scores. Time taken: {end_time - start_time:.2f}s")
        return sorted(self.result.items(), key=lambda d: d[1], reverse=True)

    @staticmethod
    def dict_add(_dict, key):
        """增加字典中 key 的计数"""
        if key in _dict:
            _dict[key] += 1
        else:
            _dict[key] = 1

el相关方法定义

## 测试读取


In [1]:
import csv
# 从 CSV 文件读取数据
with open('processed_data.csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    data = [row for row in reader]

In [2]:
import json
from tqdm import tqdm
import time
from collections import defaultdict
import pickle
import os

class CompKeyG:

    def __init__(self, name):
        # 初始化字典和变量
        self.name = name
        self.inters = {}         # 中介关键词字典
        self.comps = {}          # 竞争关键词字典
        self.inters_all = {}     # 每个中介关键词出现的总次数
        self.result = {}         # 存放最终竞争度排名结果
        self.keyword = ''        # 当前处理的种子关键词
        self.num_queries = 0     # 当前种子关键词的查询数量
        self.num_inters = 0      # 中介关键词数量
        self.threshold = 0       # 中介关键词过滤阈值
        self.comp_cache = {}     # 缓存关键词竞争度排名结果

        # 图结构
        self.word_counts = {}    # 每个词在所有查询中出现的次数
        self.co_occurrence = {}  # 词与词之间的共现次数

    def initialize(self, keyword):
        """初始化关键词相关数据"""
        self.inters.clear()
        self.comps.clear()
        self.inters_all.clear()
        self.result.clear()
        self.num_queries = 0
        self.num_inters = 0
        self.keyword = keyword

    def build_graph(self, data):
        """构建词共现图"""
        print("Building word co-occurrence graph...")
        start_time = time.time()
        word_counts = defaultdict(int)
        co_occurrence = defaultdict(lambda: defaultdict(int))

        for query in data:
            unique_words = set(query)
            # 更新每个词的出现次数
            for word in unique_words:
                word_counts[word] += 1
            # 更新词之间的共现次数
            for word1 in unique_words:
                for word2 in unique_words:
                    if word1 != word2:
                        co_occurrence[word1][word2] += 1

        self.word_counts = dict(word_counts)
        self.co_occurrence = {k: dict(v) for k, v in co_occurrence.items()}
        end_time = time.time()
        print(f"Graph built. Time taken: {end_time - start_time:.2f}s")

    def save_graph(self, filename="graph.pkl"):
        """将图结构保存到磁盘"""
        print(f"Saving graph to '{filename}'...")
        try:
            with open(filename, 'wb') as f:
                pickle.dump({'word_counts': self.word_counts, 'co_occurrence': self.co_occurrence}, f)
            print("Graph saved successfully.")
        except Exception as e:
            print(f"An error occurred while saving the graph: {e}")

    def load_graph(self, filename="graph.pkl"):
        """从磁盘加载图结构"""
        print(f"Loading graph from '{filename}'...")
        try:
            with open(filename, 'rb') as f:
                data = pickle.load(f)
                self.word_counts = data['word_counts']
                self.co_occurrence = data['co_occurrence']
            print("Graph loaded successfully.")
            # 统计节点数量
            num_nodes = len(self.word_counts)
            
            # 统计边的数量
            num_edges = sum(len(neighbors) for neighbors in self.co_occurrence.values())
    
            print("Graph loaded successfully.")
            print(f"Graph has {num_nodes} nodes and {num_edges} edges.")
            return True
        except FileNotFoundError:
            print(f"Graph file '{filename}' not found.")
            return False
        except Exception as e:
            print(f"An error occurred while loading the graph: {e}")
            return False

    def train(self, data, keywords: list, num_results: int = 20, graph_filename="graph.pkl"):
        """训练模型，对每个关键词进行竞争性分析"""
        # 尝试从磁盘加载图结构
        graph_loaded = self.load_graph(graph_filename)
        if not graph_loaded:
            # 构建词共现图
            self.build_graph(data)
            # 将图结构保存到磁盘
            self.save_graph(graph_filename)

        for step, keyword in enumerate(keywords):
            print(f"Training keyword {step + 1}/{len(keywords)}: '{keyword}'")
            # 初始化
            self.initialize(keyword)
            # 筛选包含种子关键词的查询数量
            self.num_queries = self.word_counts.get(keyword, 0)
            if self.num_queries == 0:
                print(f"No queries found containing keyword '{keyword}'. Skipping...")
                continue
            # 分析中介关键词和竞争关键词
            self.analyze_inter()
            self.analyze_comp()
            # 计算竞争度并缓存结果
            self.comp_cache[keyword] = self.calculate_comp()[:num_results]
            print(f"Completed training for '{keyword}'")

    def save_to_disk(self, filename="comp_cache.json"):
        """将训练结果保存到磁盘上的文件中"""
        try:
            # 打开文件并以 JSON 格式写入 comp_cache
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(self.comp_cache, f, ensure_ascii=False, indent=4)
            print(f"Training results saved successfully to '{filename}'")
        except Exception as e:
            print(f"An error occurred while saving to disk: {e}")

    def predict(self, keyword):
        """预测指定关键词的竞争度排名"""
        return self.comp_cache.get(keyword, None)

    def analyze_inter(self):
        """分析中介关键词"""
        print("Analyzing intermediate keywords (inters)...")
        start_time = time.time()
        # 获取与种子关键词共现的所有词汇及其共现次数
        inters = self.co_occurrence.get(self.keyword, {})
        # 过滤出现次数较少的中介关键词
        self.inters = {word: count for word, count in inters.items() if count >= self.threshold}
        self.num_inters = len(self.inters)
        end_time = time.time()
        print(f"Got {self.num_inters}/{len(inters)} inters after filtering. Time taken: {end_time - start_time:.2f}s")

    def analyze_comp(self):
        """分析竞争关键词"""
        print("Analyzing competing keywords (comps)...")
        start_time = time.time()
        with tqdm(total=self.num_inters, desc="Progress", unit="keyword") as pbar:
            for inter in self.inters:
                # 获取中介关键词的总出现次数
                self.inters_all[inter] = self.word_counts.get(inter, 0)
                # 获取与中介关键词共现的词汇及其共现次数
                comps = self.co_occurrence.get(inter, {})
                for comp, count in comps.items():
                    if comp != self.keyword:
                        key = (comp, inter)
                        self.comps[key] = self.comps.get(key, 0) + count
                pbar.update(1)
        end_time = time.time()
        print(f"Completed analyzing comps. Total comp-inter pairs: {len(self.comps)}. Time taken: {end_time - start_time:.2f}s")

    def calculate_comp(self):
        """计算竞争关键词的竞争度"""
        print("Calculating competition scores...")
        start_time = time.time()
        for (comp, inter), num in self.comps.items():
            value = self.result.get(comp, 0)
            # 跳过没有竞争关键词的中介关键词
            if self.inters_all[inter] == self.inters[inter]:
                continue  # 跳过该中介关键词

            # 计算竞争度分数
            self.result[comp] = value + (self.inters[inter] / self.num_queries) * (num / (self.inters_all[inter] - self.inters[inter]))
        end_time = time.time()
        print(f"Completed calculation of competition scores. Time taken: {end_time - start_time:.2f}s")
        return sorted(self.result.items(), key=lambda d: d[1], reverse=True)

# 运行模型

In [9]:
# 定义要分析的关键词列表
keywords = ["苹果"]

# 创建 CompKey 实例，指定模型名称
comp_key_model = CompKeyG(name="KeywordCompetitionModel")

# 调用 train 方法进行训练，每个关键词返回前 10 个竞争关键词
comp_key_model.train(data=data, keywords=keywords, num_results=10,graph_filename="old_graph.pkl")
comp_key_model.save_to_disk("comp_cache.json")

# 对某个关键词预测竞争度排名结果
keyword_to_predict = "苹果"
competition_results = comp_key_model.predict(keyword_to_predict)

# 输出预测结果
if competition_results:
    print(f"竞争关键词排名结果 for '{keyword_to_predict}':")
    for rank, (comp_word, score) in enumerate(competition_results, 1):
        print(f"{rank}. {comp_word}: {score}")
else:
    print(f"No competition data found for keyword: {keyword_to_predict}")

Loading graph from 'old_graph.pkl'...
Graph loaded successfully.
Graph loaded successfully.
Graph has 835458 nodes and 42071568 edges.
Training keyword 1/1: '苹果'
Analyzing intermediate keywords (inters)...
Got 11191/11191 inters after filtering. Time taken: 0.00s
Analyzing competing keywords (comps)...


Progress: 100%|██████████| 11191/11191 [00:14<00:00, 796.20keyword/s] 


Completed analyzing comps. Total comp-inter pairs: 19093854. Time taken: 14.06s
Calculating competition scores...
Completed calculation of competition scores. Time taken: 10.12s
Completed training for '苹果'
Training results saved successfully to 'comp_cache.json'
竞争关键词排名结果 for '苹果':
1. 的: 0.5793444461260963
2. 怎么: 0.29552916311292643
3. 什么: 0.26697918611865296
4. 吗: 0.2512370447773059
5. 手机: 0.20852050250510326
6. 多少: 0.2067247086753324
7. 是: 0.20130953353996792
8.  : 0.17544105076919073
9. 和: 0.1600091529151374
10. 了: 0.1536104244394003


# 下面是尝试过的用处不大的代码

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

# 初始化 Elasticsearch 连接
es = Elasticsearch('http://localhost:9200',request_timeout=60 ) # 设置超时时间为 60 秒)
index_name = 'queries_index'

def initialize_es_index():
    """初始化 Elasticsearch 索引，定义映射"""
    mapping = {
        "mappings": {
            "properties": {
                "query": {
                    "type": "text",
                    "analyzer": "standard"
                }
            }
        }
    }
    if not es.indices.exists(index=index_name):
        es.indices.create(index=index_name, body=mapping)

def index_data_to_es(data):
    """将数据批量索引到 Elasticsearch"""
    actions = [
        {
            "_index": index_name,
            "_source": {
                "query": ' '.join(query)  # 将关键词列表转换为字符串
            }
        }
        for query in data
    ]
    bulk(es, actions)
    print("Data indexing completed.")

def search_queries_by_keyword_es(keyword):
    """在 Elasticsearch 中搜索包含指定关键词的查询"""
    body = {
        "query": {
            "match": {
                "query": keyword
            }
        },
        "_source": ["query"]  # 仅获取查询字段
    }
    # 使用 scroll API 获取所有匹配的文档
    results = []
    page = es.search(index=index_name, body=body, scroll='2m', size=1000)
    sid = page['_scroll_id']
    scroll_size = len(page['hits']['hits'])
    while scroll_size > 0:
        hits = page['hits']['hits']
        for hit in hits:
            query = hit['_source']['query'].split()
            results.append(query)
        page = es.scroll(scroll_id=sid, scroll='2m')
        sid = page['_scroll_id']
        scroll_size = len(page['hits']['hits'])
    return results

In [ ]:
from tqdm import tqdm
import time
from collections import defaultdict

class CompKeyEL:

    def __init__(self, name):
        # 初始化字典和变量
        self.name = name
        self.inters = {}         # 中介关键词字典
        self.comps = {}          # 竞争关键词字典
        self.inters_all = {}     # 每个中介关键词出现的总次数
        self.result = {}         # 存放最终竞争度排名结果
        self.keyword = ''        # 当前处理的种子关键词
        self.num_queries = 0     # 当前种子关键词的查询数量
        self.num_inters = 0      # 中介关键词数量
        self.threshold = 0       # 中介关键词过滤阈值
        self.comp_cache = {}     # 缓存关键词竞争度排名结果
    
    def initialize(self, keyword):
        """初始化关键词相关数据"""
        self.inters.clear()
        self.comps.clear()
        self.inters_all.clear()
        self.result.clear()
        self.num_queries = 0
        self.num_inters = 0
        self.keyword = keyword

    def train(self, data, keywords: list, num_results: int = 20):
        """训练模型，对每个关键词进行竞争性分析"""
        # 初始化 Elasticsearch 索引，并将数据索引到 Elasticsearch
        print("Indexing data into Elasticsearch...")
        for step, keyword in enumerate(keywords):
            print(f"Training keyword {step + 1}/{len(keywords)}: '{keyword}'")
            # 初始化
            self.initialize(keyword)
            # 使用 Elasticsearch 查询包含种子关键词的查询
            filtered_data = search_queries_by_keyword_es(keyword)
            self.num_queries = len(filtered_data)
            # 分析中介关键词和竞争关键词
            self.analyze_inter(filtered_data)
            self.analyze_comp()
            # 计算竞争度并缓存结果
            self.comp_cache[keyword] = self.calculate_comp()[:num_results]
            print(f"Completed training for '{keyword}'")

    def analyze_inter(self, filtered_data):
        """分析中介关键词"""
        print("Analyzing intermediate keywords (inters)...")
        start_time = time.time()
        for query in filtered_data:
            for word in query:
                if word != self.keyword:
                    self.dict_add(self.inters, word)
        inters_total = len(self.inters)
        # 过滤出现次数较少的中介关键词
        keys_to_delete = [key for key, count in self.inters.items() if count < self.threshold]
        for key in keys_to_delete:
            del self.inters[key]
        self.num_inters = len(self.inters)
        end_time = time.time()
        print(f"Got {self.num_inters}/{inters_total} inters after filtering. Time taken: {end_time - start_time:.2f}s")

    def analyze_comp(self):
        """分析竞争关键词"""
        print("Analyzing competing keywords (comps)...")
        start_time = time.time()
        with tqdm(total=self.num_inters, desc="Progress", unit="keyword") as pbar:
            for step, key in enumerate(self.inters):
                query_start_time = time.time()
                # 使用 Elasticsearch 查询包含中介关键词的查询
                inter_queries = search_queries_by_keyword_es(key)
                self.inters_all[key] = len(inter_queries)
                for query in inter_queries:
                    for word in query:
                        if word != key and word != self.keyword:
                            self.dict_add(self.comps, (word, key))
                query_end_time = time.time()
                # 更新进度条
                pbar.set_postfix(time_per_keyword=f"{query_end_time - query_start_time:.2f}s")
                pbar.update(1)
        end_time = time.time()
        print(f"Completed analyzing comps. Total comp-inter pairs: {len(self.comps)}. Time taken: {end_time - start_time:.2f}s")

    def calculate_comp(self):
        """计算竞争关键词的竞争度"""
        print("Calculating competition scores...")
        start_time = time.time()
        for comp, inter in self.comps:
            num = self.comps[(comp, inter)]
            value = self.result.get(comp, 0)
            # 跳过没有竞争关键词的中介关键词
            if self.inters_all[inter] == self.inters[inter]:
                continue  # 跳过该中介关键词

            # 计算竞争度分数
            self.result[comp] = value + (self.inters[inter] / self.num_queries) * (num / (self.inters_all[inter] - self.inters[inter]))
        
        end_time = time.time()
        print(f"Completed calculation of competition scores. Time taken: {end_time - start_time:.2f}s")
        return sorted(self.result.items(), key=lambda d: d[1], reverse=True)

    @staticmethod
    def dict_add(_dict, key):
        """增加字典中 key 的计数"""
        if key in _dict:
            _dict[key] += 1
        else:
            _dict[key] = 1

In [ ]:
initialize_es_index()
index_data_to_es(data)

In [ ]:
from collections import Counter
# 统计词频
word_counts = Counter(word for query in data for word in query)

# 获取频率中等的10个词
# 首先排序，然后取中间10个
sorted_words = sorted(word_counts.items(), key=lambda item: item[1])
middle_index = len(sorted_words) // 2
middle_10_words = sorted_words[middle_index - 5: middle_index + 5]  # 取中间10个
top_20_words = sorted_words[-200:-100]  # 取频率最高的10个
bottom_10_words = sorted_words[:10]  # 取频率最低的10个

# 输出结果
print("Top 10 Words (Highest Frequency):")
for word, freq in top_20_words:
    print(f"{word}: {freq}")

# print("\nBottom 10 Words (Lowest Frequency):")
# for word, freq in bottom_10_words:
#     print(f"{word}: {freq}")
# 
# print("\nMiddle 10 Words (Medium Frequency):")
# for word, freq in middle_10_words:
#     print(f"{word}: {freq}")
# 